<a href="https://colab.research.google.com/github/Munazza-Farees/NITW-SIP2025-Project/blob/main/AKN_FGD_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pywt
from sklearn.metrics import classification_report, roc_curve, f1_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from scipy.spatial.distance import euclidean
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
from scipy.stats import pearsonr
from collections import Counter


In [ ]:
# Load dataset
URL = '/content/drive/MyDrive/Colab Notebooks/Client1 (copy).csv'
data = pd.read_csv(URL)

In [ ]:
# Input missing values
data.fillna(data.median(numeric_only=True), inplace=True)
data['Protocol'].fillna(data['Protocol'].mode()[0], inplace=True)
data['Flags'].fillna(data['Flags'].mode()[0], inplace=True)

# Convert types and encode categorical features
data = data.astype({'Time':'float', 'Label': int})
data = pd.get_dummies(data, columns=['Protocol', 'Flags'], prefix=['Protocol', 'Flags'])
data = data.drop(columns=['Source', 'Destination'], errors='ignore')

<ipython-input-78-890462510>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Protocol'].fillna(data['Protocol'].mode()[0], inplace=True)
<ipython-input-78-890462510>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)

In [ ]:
# Sequence Alignment with Smith-Waterman Algorithm
def smith_waterman(seq1, seq2, match_score=2, mismatch_score=-2, gap_penalty=-1):
    n, m = len(seq1), len(seq2)
    H = np.zeros((n+1, m+1))
    for i in range(1, n+1):
        for j in range(1, m+1):
            match = H[i-1, j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_score)
            delete = H[i-1, j] + gap_penalty
            insert = H[i, j-1] + gap_penalty
            H[i, j] = max(match, delete, insert, 0)
    return np.max(H)

# Construct benchmark sequences for RTO values
rto_values = [1, 2]
benchmark_sequences = {rto: np.array([1 if i % rto < 0.1 else 0 for i in np.arange(0, 60, 0.1)]) for rto in rto_values}

In [ ]:
# Estimate DU duration using UDP packet counts
cw_duration = 60
data['CW_ID'] = ((data['Time'] - data['Time'].min()) // cw_duration).astype(int)
udp_data = data[data['Protocol_UDP'] == 1]  # Assuming one-hot encoded 'Protocol_UDP'
cw_udp_counts = udp_data.groupby('CW_ID')['Length'].count().reindex(data['CW_ID'].unique(), fill_value=0)
cw_udp_seq = cw_udp_counts.values

max_score = -np.inf
time_du = 1
for rto, seq in benchmark_sequences.items():
    score = smith_waterman(cw_udp_seq[:len(seq)], seq)
    if score > max_score:
        max_score = score
        time_du = rto

# Create DUs based on estimated TimeDU
data['DU_ID'] = ((data['Time'] - data['Time'].min()) // time_du).astype(int)
print("Estimated DU Duration:", time_du, "seconds")
print("Packets per DU:\n", data.groupby('DU_ID').size().describe())

Estimated DU Duration: 2 seconds
Packets per DU:
 count    733.000000
mean      47.553888
std      131.489508
min        2.000000
25%        8.000000
50%       14.000000
75%       24.000000
max      733.000000
dtype: float64


In [ ]:
# Feature Aggregation
data['TCP_Packets'] = data['Protocol_TCP']  # Assuming one-hot encoded 'Protocol_TCP'
data['Total_Packets'] = 1   # Each row is a packet
data['Burstiness'] = data.groupby('DU_ID')['Packet_Rate'].transform(lambda x:x.max() / (x.mean() + 1e-10))

features_to_aggregate = [
    'Length', 'Inter_Arrival_Time', 'Connection_Duration', 'Packet_Rate',
    'Flow_Bytes_Per_Second', 'Flow_Packets_Per_Second', 'Forward_Packets',
    'Backward_Packets', 'Ratio_Fwd_Bwd', 'Entropy', 'Packet_Size_Variance',
    'Burstiness', 'TCP_Packets', 'Total_Packets'
]

agg_funcs = {col: ['mean', 'std', 'max', 'min'] for col in features_to_aggregate}
agg_funcs['Label'] = lambda x: 1 if (x == 1).mean() >= 0.05 else 0
agg_data = data.groupby('DU_ID').agg(agg_funcs)
agg_data.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in agg_data.columns]
agg_data.reset_index(inplace=True)

In [ ]:
# Compute wavelet packet entropy per DU
def compute_du_entropy(group):
    signal = group['Length'].values
    if len(signal) < 2:
        return 0
    wp = pywt.WaveletPacket(data=signal, wavelet='db1', mode='symmetric', maxlevel=3)
    energies = [np.sum(np.square(node.data)) for node in wp.get_level(3)]
    total_energies = np.sum(energies)
    probs = np.array(energies) / (total_energies + 1e-10)
    return -np.sum(probs * np.log2(probs + 1e-10))

agg_data['Entropy'] = data.groupby('DU_ID').apply(compute_du_entropy).reindex(agg_data.index).values

<ipython-input-82-2986227820>:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg_data['Entropy'] = data.groupby('DU_ID').apply(compute_du_entropy).reindex(agg_data.index).values


In [ ]:
# Feature scaling

agg_data_features = agg_data.drop(columns=['DU_ID', 'Label_<lambda>'])

agg_data_features.fillna(0, inplace=True)

for col in agg_data_features.columns:
    if agg_data_features[col].dtype == 'bool':
        agg_data_features[col] = agg_data_features[col].astype(int)


for col in agg_data_features.columns:
    p1, p99 = agg_data_features[col].quantile([0.01, 0.99])
    agg_data_features[col] = agg_data_features[col].clip(p1, p99)

scaler = RobustScaler()
X_scaled = scaler.fit_transform(agg_data_features)
X_scaled_data = pd.DataFrame(X_scaled, columns=agg_data_features.columns)


X_scaled_data['DU_ID'] = agg_data['DU_ID']
# X_scaled_data['Entropy_DU'] = X_scaled_data['DU_ID'].map(agg_data['Entropy'])
X_scaled_data['Label'] = agg_data['Label_<lambda>']
X_scaled_data.fillna(0, inplace=True)

# Balance DUs with SMOTE
X_array = X_scaled_data.drop(columns=['DU_ID', 'Label']).values
y_array = X_scaled_data['Label'].values
smote = SMOTE(random_state=42)

X_resampled, y_resampled = smote.fit_resample(X_array, y_array)
X_scaled_data_resampled = pd.DataFrame(X_resampled, columns=X_scaled_data.drop(columns=['DU_ID', 'Label']).columns)

X_scaled_data_resampled['Label'] = y_resampled
# X_scaled_data_resampled['DU_ID'] = X_scaled_data['DU_ID'].iloc[:len(X_resampled)].values
# X_scaled_data_resampled['Entropy_DU'] = X_scaled_data_resampled['DU_ID'].map(agg_data['Entropy'])

In [ ]:
# AKN: Calculate Neurons' Number (Algorithm 1)
def calculate_optimal_neurons(X, threshold_ratio=0.5):
    centers = [X[0]]
    max_dist = 0
    second = X[1]
    for row in X:
        dist = euclidean(row, centers[0])
        if dist > max_dist:
            max_dist = dist
            second = row
    centers.append(second)
    while True:
        min_dists = [min([euclidean(row, center) for center in centers]) for row in X]
        D_max = max(min_dists)
        if D_max > threshold_ratio * euclidean(centers[0], centers[1]):
            new_center = X[np.argmax(min_dists)]
            centers.append(new_center)
        else:
            break
    return len(centers), np.array(centers)

# AKN: Calculate Initial Weights (Algorithm 2)
def calculate_initial_weights(R, numcl):
    N = (R - R.min(axis=0)) / (R.max(axis=0) - R.min(axis=0) + 1e-10)
    C = N.mean(axis=0)
    dmax = np.max([euclidean(n, C) for n in N])
    X = np.zeros((R.shape[1], numcl))
    for j in range(numcl):
        X[:, j] = C + np.random.uniform(-dmax, dmax, R.shape[1])
    return X.T

# AKN: Clustering (Algorithm 3)
def kohonen_clustering(X, initial_centers, num_epochs=100, initial_lr=0.5, initial_radius=0.5):
    weights = initial_centers.copy()
    num_neurons = len(weights)
    sigma = initial_radius * num_neurons
    sigma_decay = sigma / num_epochs
    for epoch in range(num_epochs):
        lr = initial_lr * np.exp(-epoch / num_epochs)
        sigma = sigma - sigma_decay * epoch
        for x in X:
            distances = euclidean_distances([x], weights).flatten()
            winner_idx = np.argmin(distances)
            for j in range(num_neurons):
                dist_to_winner = np.abs(j - winner_idx)
                influence = np.exp(-dist_to_winner**2 / (2 * sigma**2))
                weights[j] += lr * influence * (x - weights[j])
    assignments = [np.argmin(euclidean_distances([x], weights)) for x in X]
    return assignments, weights

In [ ]:
# Apply AKN
num_neurons, initial_centers = calculate_optimal_neurons(X_resampled, threshold_ratio=0.5)
initial_weights = calculate_initial_weights(X_resampled, num_neurons)
cluster_labels, final_weights = kohonen_clustering(X_resampled, initial_weights, num_epochs=100)
X_scaled_data_resampled['Cluster'] = cluster_labels
print("Clustering Silhouette Score:", silhouette_score(X_resampled, cluster_labels))

Clustering Silhouette Score: 0.7575332247943308


In [ ]:
X_scaled_data_resampled['DU_ID'] = X_scaled_data['DU_ID'].iloc[:len(X_scaled_data_resampled)].values
X_scaled_data_resampled['Entropy_DU'] = X_scaled_data_resampled['DU_ID'].map(agg_data['Entropy'])

# Compute MAD
mean_C = (X_scaled_data_resampled['TCP_Packets_mean'] / (X_scaled_data_resampled['TCP_Packets_mean'] + 1e-10)).mean()
std_C = (X_scaled_data_resampled['TCP_Packets_mean'] / (X_scaled_data_resampled['TCP_Packets_mean'] + 1e-10)).std()
mean_H = X_scaled_data_resampled['Entropy_DU'].mean()
std_H = X_scaled_data_resampled['Entropy_DU'].std()
mean_P = X_scaled_data_resampled.apply(lambda row: pearsonr(row['TCP_Packets_mean'], row['Total_Packets_mean'])[0] if row['Total_Packets_mean'] > 0 else 0, axis=1).mean()
std_P = X_scaled_data_resampled.apply(lambda row: pearsonr(row['TCP_Packets_mean'], row['Total_Packets_mean'])[0] if row['Total_Packets_mean'] > 0 else 0, axis=1).std()

mad_scores = []
for _, row in X_scaled_data_resampled.iterrows():
    C = (row['TCP_Packets_mean'] / (row['TCP_Packets_mean'] + 1e-10) - mean_C) / (std_C + 1e-10)
    H = (row['Entropy_DU'] - mean_H) / (std_H + 1e-10)
    P = (pearsonr([row['TCP_Packets_mean']], [row['Total_Packets_mean']])[0] - mean_P) / (std_P + 1e-10) if row['Total_Packets_mean'] > 0 else 0
    mad = np.sqrt(0.4 * C**2 + 0.3 * H**2 + 0.3 * P**2)
    mad_scores.append(mad)
X_scaled_data_resampled['MAD'] = mad_scores
X_scaled_data_resampled['MAD'] = scaler.fit_transform(X_scaled_data_resampled[['MAD']]).flatten()

# Optimize Thresholds
fpr, tpr, thresholds = roc_curve(X_scaled_data_resampled['Label'], X_scaled_data_resampled['MAD'])
optimal_idx = np.argmax(tpr - fpr)
optimal_z = (thresholds[optimal_idx] - X_scaled_data_resampled['MAD'].mean()) / X_scaled_data_resampled['MAD'].std()
LMAD = X_scaled_data_resampled['MAD'].mean() + optimal_z * X_scaled_data_resampled['MAD'].std()
X_scaled_data_resampled['Abnormal_DU'] = (X_scaled_data_resampled['MAD'] > LMAD).astype(int)

best_ladur = 0.55
best_f1 = 0
for ladur in [0.5, 0.55, 0.6]:
    cluster_votes = X_scaled_data_resampled.groupby('Cluster')['Abnormal_DU'].mean()
    abnormal_clusters = cluster_votes[cluster_votes >= ladur].index
    X_scaled_data_resampled['Predicted_Label'] = X_scaled_data_resampled['Cluster'].apply(lambda c: 1 if c in abnormal_clusters else 0)
    f1 = f1_score(X_scaled_data_resampled['Label'], X_scaled_data_resampled['Predicted_Label'])
    if f1 > best_f1:
        best_f1 = f1
        best_ladur = ladur

# Final Detection
LMAD = X_scaled_data_resampled['MAD'].mean() + optimal_z * X_scaled_data_resampled['MAD'].std()
X_scaled_data_resampled['Abnormal_DU'] = (X_scaled_data_resampled['MAD'] > LMAD).astype(int)
cluster_votes = X_scaled_data_resampled.groupby('Cluster')['Abnormal_DU'].mean()
abnormal_clusters = cluster_votes[cluster_votes >= best_ladur].index
X_scaled_data_resampled['Predicted_Label'] = X_scaled_data_resampled['Cluster'].apply(lambda c: 1 if c in abnormal_clusters else 0)
X_scaled_data['Predicted_Label'] = X_scaled_data['DU_ID'].map(X_scaled_data_resampled.groupby('DU_ID')['Predicted_Label'].first())
print("Final Detection Evaluation:\n")
print(classification_report(X_scaled_data['Label'], X_scaled_data['Predicted_Label']))

# Cross-validate Random Forest
from sklearn.model_selection import cross_val_score
rf = RandomForestClassifier(random_state=42)
scores = cross_val_score(rf, X_array, X_scaled_data['Label'], cv=5, scoring='f1_weighted')
print("Random Forest Cross-Validation F1 Scores:", scores, "Mean:", scores.mean())

ValueError: Length of values (733) does not match length of index (1390)

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pywt
from sklearn.metrics import classification_report, roc_curve, f1_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from scipy.spatial.distance import euclidean
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
from scipy.stats import pearsonr
from collections import Counter
from sklearn.neighbors import NearestNeighbors

# Load dataset
URL = '/content/drive/MyDrive/Colab Notebooks/Client1 (copy).csv'
data = pd.read_csv(URL)

# Impute missing values
data.fillna(data.median(numeric_only=True), inplace=True)
data['Protocol'].fillna(data['Protocol'].mode()[0], inplace=True)
data['Flags'].fillna(data['Flags'].mode()[0], inplace=True)

# Convert types and encode categorical features
data = data.astype({'Time': 'float', 'Label': int})
data = pd.get_dummies(data, columns=['Protocol', 'Flags'], prefix=['Protocol', 'Flags'])
data = data.drop(columns=['Source', 'Destination'], errors='ignore')

# Sequence Alignment with Smith-Waterman Algorithm
def smith_waterman(seq1, seq2, match_score=2, mismatch_score=-2, gap_penalty=-1):
    n, m = len(seq1), len(seq2)
    H = np.zeros((n+1, m+1))
    for i in range(1, n+1):
        for j in range(1, m+1):
            match = H[i-1, j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_score)
            delete = H[i-1, j] + gap_penalty
            insert = H[i, j-1] + gap_penalty
            H[i, j] = max(match, delete, insert, 0)
    return np.max(H)

# Construct benchmark sequences for RTO values
rto_values = [1, 2]
benchmark_sequences = {rto: np.array([1 if i % rto < 0.1 else 0 for i in np.arange(0, 60, 0.1)]) for rto in rto_values}

# Estimate DU duration using UDP packet counts
cw_duration = 60
data['CW_ID'] = ((data['Time'] - data['Time'].min()) // cw_duration).astype(int)
udp_data = data[data['Protocol_UDP'] == 1]  # Assuming one-hot encoded 'Protocol_UDP'
cw_udp_counts = udp_data.groupby('CW_ID')['Length'].count().reindex(data['CW_ID'].unique(), fill_value=0)
cw_udp_seq = cw_udp_counts.values

max_score = -np.inf
time_du = 1
for rto, seq in benchmark_sequences.items():
    score = smith_waterman(cw_udp_seq[:len(seq)], seq)
    if score > max_score:
        max_score = score
        time_du = rto

# Create DUs based on estimated TimeDU
data['DU_ID'] = ((data['Time'] - data['Time'].min()) // time_du).astype(int)
print("Estimated DU Duration:", time_du, "seconds")
print("Packets per DU:\n", data.groupby('DU_ID').size().describe())

# Feature Aggregation
data['TCP_Packets'] = data['Protocol_TCP']
data['Total_Packets'] = 1  # Each row is a packet
data['Burstiness'] = data.groupby('DU_ID')['Packet_Rate'].transform(lambda x: x.max() / (x.mean() + 1e-10))

features_to_aggregate = [
    'Length', 'Inter_Arrival_Time', 'Connection_Duration', 'Packet_Rate',
    'Flow_Bytes_Per_Second', 'Flow_Packets_Per_Second', 'Forward_Packets',
    'Backward_Packets', 'Ratio_Fwd_Bwd', 'Entropy', 'Packet_Size_Variance',
    'Burstiness', 'TCP_Packets', 'Total_Packets'
]

agg_funcs = {col: ['mean', 'std', 'max', 'min'] for col in features_to_aggregate}
agg_funcs['Label'] = lambda x: 1 if (x == 1).mean() >= 0.05 else 0
agg_data = data.groupby('DU_ID').agg(agg_funcs)
agg_data.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in agg_data.columns]
agg_data.reset_index(inplace=True)

# Compute wavelet packet entropy per DU
def compute_du_entropy(group):
    signal = group['Length'].values
    if len(signal) < 2:
        return 0
    wp = pywt.WaveletPacket(data=signal, wavelet='db1', mode='symmetric', maxlevel=3)
    energies = [np.sum(np.square(node.data)) for node in wp.get_level(3)]
    total_energy = np.sum(energies)
    probs = np.array(energies) / (total_energy + 1e-10)
    return -np.sum(probs * np.log2(probs + 1e-10))

agg_data['Entropy_DU'] = data.groupby('DU_ID').apply(compute_du_entropy).reindex(agg_data.index).values

# Feature Scaling
agg_data_features = agg_data.drop(columns=['DU_ID', 'Label_<lambda>'])
agg_data_features.fillna(0, inplace=True)

for col in agg_data_features.columns:
    if agg_data_features[col].dtype == 'bool':
        agg_data_features[col] = agg_data_features[col].astype(int)

for col in agg_data_features.columns:
    p1, p99 = agg_data_features[col].quantile([0.01, 0.99])
    agg_data_features[col] = agg_data_features[col].clip(p1, p99)

scaler = RobustScaler()
X_scaled = scaler.fit_transform(agg_data_features)
X_scaled_data = pd.DataFrame(X_scaled, columns=agg_data_features.columns)
X_scaled_data['DU_ID'] = agg_data['DU_ID'].values
X_scaled_data['Label'] = agg_data['Label_<lambda>'].values
X_scaled_data['Entropy_DU'] = agg_data['Entropy_DU'].values
X_scaled_data.fillna(0, inplace=True)

# Balance DUs with SMOTE and propagate DU_ID
X_array = X_scaled_data.drop(columns=['DU_ID', 'Label', 'Entropy_DU']).values
y_array = X_scaled_data['Label'].values
smote = SMOTE(random_state=42, k_neighbors=5)
X_resampled, y_resampled = smote.fit_resample(X_array, y_array)

# Find nearest neighbors to assign DU_ID to synthetic samples
nn = NearestNeighbors(n_neighbors=1).fit(X_array)
distances, indices = nn.kneighbors(X_resampled)
du_ids_resampled = X_scaled_data['DU_ID'].iloc[indices.flatten()].values
entropy_du_resampled = X_scaled_data['Entropy_DU'].iloc[indices.flatten()].values

X_scaled_data_resampled = pd.DataFrame(X_resampled, columns=X_scaled_data.drop(columns=['DU_ID', 'Label', 'Entropy_DU']).columns)
X_scaled_data_resampled['Label'] = y_resampled
X_scaled_data_resampled['DU_ID'] = du_ids_resampled
X_scaled_data_resampled['Entropy_DU'] = entropy_du_resampled

<ipython-input-1-3086762825>:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Protocol'].fillna(data['Protocol'].mode()[0], inplace=True)
<ipython-input-1-3086762825>:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tru

Estimated DU Duration: 2 seconds
Packets per DU:
 count    733.000000
mean      47.553888
std      131.489508
min        2.000000
25%        8.000000
50%       14.000000
75%       24.000000
max      733.000000
dtype: float64


<ipython-input-1-3086762825>:98: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg_data['Entropy_DU'] = data.groupby('DU_ID').apply(compute_du_entropy).reindex(agg_data.index).values


In [ ]:
# AKN: Calculate Neurons' Number (Algorithm 1)
def calculate_optimal_neurons(X, threshold_ratio=0.5):
    centers = [X[0]]
    max_dist = 0
    second = X[1]
    for row in X:
        dist = euclidean(row, centers[0])
        if dist > max_dist:
            max_dist = dist
            second = row
    centers.append(second)
    while True:
        min_dists = [min([euclidean(row, center) for center in centers]) for row in X]
        D_max = max(min_dists)
        if D_max > threshold_ratio * euclidean(centers[0], centers[1]):
            new_center = X[np.argmax(min_dists)]
            centers.append(new_center)
        else:
            break
    return len(centers), np.array(centers)

# AKN: Calculate Initial Weights (Algorithm 2)
def calculate_initial_weights(R, numcl):
    N = (R - R.min(axis=0)) / (R.max(axis=0) - R.min(axis=0) + 1e-10)
    C = N.mean(axis=0)
    dmax = np.max([euclidean(n, C) for n in N])
    X = np.zeros((R.shape[1], numcl))
    for j in range(numcl):
        X[:, j] = C + np.random.uniform(-dmax, dmax, R.shape[1])
    return X.T

# AKN: Clustering (Algorithm 3)
def kohonen_clustering(X, initial_centers, num_epochs=100, initial_lr=0.5, initial_radius=0.5):
    weights = initial_centers.copy()
    num_neurons = len(weights)
    sigma = initial_radius * num_neurons
    sigma_decay = sigma / num_epochs
    for epoch in range(num_epochs):
        lr = initial_lr * np.exp(-epoch / num_epochs)
        sigma = sigma - sigma_decay * epoch
        for x in X:
            distances = euclidean_distances([x], weights).flatten()
            winner_idx = np.argmin(distances)
            for j in range(num_neurons):
                dist_to_winner = np.abs(j - winner_idx)
                influence = np.exp(-dist_to_winner**2 / (2 * sigma**2))
                weights[j] += lr * influence * (x - weights[j])
    assignments = [np.argmin(euclidean_distances([x], weights)) for x in X]
    return assignments, weights


In [ ]:
# Apply AKN
X_clustering = X_scaled_data_resampled.drop(columns=['Label', 'DU_ID', 'Entropy_DU']).values
num_neurons, initial_centers = calculate_optimal_neurons(X_clustering, threshold_ratio=0.5)
initial_weights = calculate_initial_weights(X_clustering, num_neurons)
cluster_labels, final_weights = kohonen_clustering(X_clustering, initial_weights, num_epochs=100)
X_scaled_data_resampled['Cluster'] = cluster_labels
print("Clustering Silhouette Score:", silhouette_score(X_clustering, cluster_labels))

Clustering Silhouette Score: 0.7575332822833021


In [ ]:
    # Compute MAD
mean_C = (X_scaled_data_resampled['TCP_Packets_mean'] / (X_scaled_data_resampled['TCP_Packets_mean'] + 1e-10)).mean()
std_C = (X_scaled_data_resampled['TCP_Packets_mean'] / (X_scaled_data_resampled['TCP_Packets_mean'] + 1e-10)).std()
mean_H = X_scaled_data_resampled['Entropy_DU'].mean()
std_H = X_scaled_data_resampled['Entropy_DU'].std()
# Compute Pearson correlation per DU
pearson_coeffs = []
for du_id in X_scaled_data_resampled['DU_ID'].unique():
    du_data = data[data['DU_ID'] == du_id]
    if len(du_data) > 1 and du_data['Total_Packets'].sum() > 0:
        corr, _ = pearsonr(du_data['TCP_Packets'], du_data['Total_Packets'])
        pearson_coeffs.append(corr)
    else:
        pearson_coeffs.append(0)
pearson_map = dict(zip(X_scaled_data_resampled['DU_ID'].unique(), pearson_coeffs))
X_scaled_data_resampled['Pearson_Corr'] = X_scaled_data_resampled['DU_ID'].map(pearson_map)
mean_P = X_scaled_data_resampled['Pearson_Corr'].mean()
std_P = X_scaled_data_resampled['Pearson_Corr'].std()

mad_scores = []
for _, row in X_scaled_data_resampled.iterrows():
    C = (row['TCP_Packets_mean'] / (row['TCP_Packets_mean'] + 1e-10) - mean_C) / (std_C + 1e-10)
    H = (row['Entropy_DU'] - mean_H) / (std_H + 1e-10)
    P = (row['Pearson_Corr'] - mean_P) / (std_P + 1e-10) if std_P > 1e-10 else 0
    mad = np.sqrt(0.4 * C**2 + 0.3 * H**2 + 0.3 * P**2)
    mad_scores.append(mad)
X_scaled_data_resampled['MAD'] = mad_scores
X_scaled_data_resampled['MAD'] = scaler.fit_transform(X_scaled_data_resampled[['MAD']]).flatten()

<ipython-input-4-940486787>:11: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(du_data['TCP_Packets'], du_data['Total_Packets'])


In [ ]:
# Optimize Thresholds
fpr, tpr, thresholds = roc_curve(X_scaled_data_resampled['Label'], X_scaled_data_resampled['MAD'])
optimal_idx = np.argmax(tpr - fpr)
optimal_z = (thresholds[optimal_idx] - X_scaled_data_resampled['MAD'].mean()) / X_scaled_data_resampled['MAD'].std()
LMAD = X_scaled_data_resampled['MAD'].mean() + optimal_z * X_scaled_data_resampled['MAD'].std()
X_scaled_data_resampled['Abnormal_DU'] = (X_scaled_data_resampled['MAD'] > LMAD).astype(int)

best_ladur = 0.55
best_f1 = 0
for ladur in [0.5, 0.55, 0.6]:
    cluster_votes = X_scaled_data_resampled.groupby('Cluster')['Abnormal_DU'].mean()
    abnormal_clusters = cluster_votes[cluster_votes >= ladur].index
    X_scaled_data_resampled['Predicted_Label'] = X_scaled_data_resampled['Cluster'].apply(lambda c: 1 if c in abnormal_clusters else 0)
    f1 = f1_score(X_scaled_data_resampled['Label'], X_scaled_data_resampled['Predicted_Label'])
    if f1 > best_f1:
        best_f1 = f1
        best_ladur = ladur

In [ ]:
# Final Detection
X_scaled_data_resampled['Abnormal_DU'] = (X_scaled_data_resampled['MAD'] > LMAD).astype(int)
cluster_votes = X_scaled_data_resampled.groupby('Cluster')['Abnormal_DU'].mean()
abnormal_clusters = cluster_votes[cluster_votes >= best_ladur].index
X_scaled_data_resampled['Predicted_Label'] = X_scaled_data_resampled['Cluster'].apply(lambda c: 1 if c in abnormal_clusters else 0)

# Impute any remaining NaN in Predicted_Label
X_scaled_data_resampled['Predicted_Label'] = X_scaled_data_resampled['Predicted_Label'].fillna(0).astype(int)

print("Final Detection Evaluation:\n")
print(classification_report(X_scaled_data_resampled['Label'], X_scaled_data_resampled['Predicted_Label']))

# Cross-validate Random Forest
rf = RandomForestClassifier(random_state=42)
# scores = cross_val_score(rf, X_array, X_scaled_data_resampled['Label'], cv=5, scoring='f1_weighted')
scores = cross_val_score(rf, X_resampled, y_resampled, cv=5, scoring='f1_weighted')
print("Random Forest Cross-Validation F1 Scores:", scores, "Mean:", scores.mean())

KeyError: 'MAD'

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pywt
from sklearn.metrics import classification_report, roc_curve, f1_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from scipy.spatial.distance import euclidean
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
from scipy.stats import pearsonr
from collections import Counter
from sklearn.neighbors import NearestNeighbors

# Load dataset
URL = '/content/drive/MyDrive/Colab Notebooks/Client1 (copy).csv'
data = pd.read_csv(URL)

# Impute missing values
data.fillna(data.median(numeric_only=True), inplace=True)
data['Protocol'].fillna(data['Protocol'].mode()[0], inplace=True)
data['Flags'].fillna(data['Flags'].mode()[0], inplace=True)

# Convert types and encode categorical features
data = data.astype({'Time': 'float', 'Label': int})
data = pd.get_dummies(data, columns=['Protocol', 'Flags'], prefix=['Protocol', 'Flags'])
data = data.drop(columns=['Source', 'Destination'], errors='ignore')

# Sequence Alignment with Smith-Waterman Algorithm
def smith_waterman(seq1, seq2, match_score=2, mismatch_score=-2, gap_penalty=-1):
    n, m = len(seq1), len(seq2)
    H = np.zeros((n+1, m+1))
    for i in range(1, n+1):
        for j in range(1, m+1):
            match = H[i-1, j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_score)
            delete = H[i-1, j] + gap_penalty
            insert = H[i, j-1] + gap_penalty
            H[i, j] = max(match, delete, insert, 0)
    return np.max(H)

# Construct benchmark sequences for RTO values
rto_values = [1, 2]
benchmark_sequences = {rto: np.array([1 if i % rto < 0.1 else 0 for i in np.arange(0, 60, 0.1)]) for rto in rto_values}

# Estimate DU duration using UDP packet counts
cw_duration = 60
data['CW_ID'] = ((data['Time'] - data['Time'].min()) // cw_duration).astype(int)
udp_data = data[data['Protocol_UDP'] == 1]
cw_udp_counts = udp_data.groupby('CW_ID')['Length'].count().reindex(data['CW_ID'].unique(), fill_value=0)
cw_udp_seq = cw_udp_counts.values

max_score = -np.inf
time_du = 1
for rto, seq in benchmark_sequences.items():
    score = smith_waterman(cw_udp_seq[:len(seq)], seq)
    if score > max_score:
        max_score = score
        time_du = rto

# Create DUs based on estimated TimeDU
data['DU_ID'] = ((data['Time'] - data['Time'].min()) // time_du).astype(int)
print("Estimated DU Duration:", time_du, "seconds")
print("Packets per DU:\n", data.groupby('DU_ID').size().describe())

# Feature Aggregation
data['TCP_Packets'] = data['Protocol_TCP']
data['Total_Packets'] = 1
data['Burstiness'] = data.groupby('DU_ID')['Packet_Rate'].transform(lambda x: x.max() / (x.mean() + 1e-10))

features_to_aggregate = [
    'Length', 'Inter_Arrival_Time', 'Connection_Duration', 'Packet_Rate',
    'Flow_Bytes_Per_Second', 'Flow_Packets_Per_Second', 'Forward_Packets',
    'Backward_Packets', 'Ratio_Fwd_Bwd', 'Entropy', 'Packet_Size_Variance',
    'Burstiness', 'TCP_Packets', 'Total_Packets'
]

agg_funcs = {col: ['mean', 'std', 'max', 'min'] for col in features_to_aggregate}
agg_funcs['Label'] = lambda x: 1 if (x == 1).mean() >= 0.05 else 0
agg_data = data.groupby('DU_ID').agg(agg_funcs)
agg_data.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in agg_data.columns]
agg_data.reset_index(inplace=True)

# Compute wavelet packet entropy per DU
def compute_du_entropy(group):
    signal = group['Length'].values
    if len(signal) < 2:
        return 0
    wp = pywt.WaveletPacket(data=signal, wavelet='db1', mode='symmetric', maxlevel=3)
    energies = [np.sum(np.square(node.data)) for node in wp.get_level(3)]
    total_energy = np.sum(energies)
    probs = np.array(energies) / (total_energy + 1e-10)
    return -np.sum(probs * np.log2(probs + 1e-10))

agg_data['Entropy_DU'] = data.groupby('DU_ID').apply(compute_du_entropy).reindex(agg_data.index).values

# Feature Scaling
agg_data_features = agg_data.drop(columns=['DU_ID', 'Label_<lambda>'])
agg_data_features.fillna(0, inplace=True)

for col in agg_data_features.columns:
    if agg_data_features[col].dtype == 'bool':
        agg_data_features[col] = agg_data_features[col].astype(int)

for col in agg_data_features.columns:
    p1, p99 = agg_data_features[col].quantile([0.01, 0.99])
    agg_data_features[col] = agg_data_features[col].clip(p1, p99)

scaler = RobustScaler()
X_scaled = scaler.fit_transform(agg_data_features)
X_scaled_data = pd.DataFrame(X_scaled, columns=agg_data_features.columns)
X_scaled_data['DU_ID'] = agg_data['DU_ID'].values
X_scaled_data['Label'] = agg_data['Label_<lambda>'].values
X_scaled_data['Entropy_DU'] = agg_data['Entropy_DU'].values
X_scaled_data.fillna(0, inplace=True)

# Compute Pearson correlation per DU
pearson_coeffs = []
for du_id in X_scaled_data['DU_ID'].unique():
    du_data = data[data['DU_ID'] == du_id]
    if len(du_data) > 1 and du_data['Total_Packets'].sum() > 0:
        corr, _ = pearsonr(du_data['TCP_Packets'], du_data['Total_Packets'])
        pearson_coeffs.append(corr if not np.isnan(corr) else 0)
    else:
        pearson_coeffs.append(0)
pearson_map = dict(zip(X_scaled_data['DU_ID'].unique(), pearson_coeffs))
X_scaled_data['Pearson_Corr'] = X_scaled_data['DU_ID'].map(pearson_map)

# AKN: Calculate Neurons' Number (Algorithm 1)
def calculate_optimal_neurons(X, threshold_ratio=0.5):
    centers = [X[0]]
    max_dist = 0
    second = X[1]
    for row in X:
        dist = euclidean(row, centers[0])
        if dist > max_dist:
            max_dist = dist
            second = row
    centers.append(second)
    while True:
        min_dists = [min([euclidean(row, center) for center in centers]) for row in X]
        D_max = max(min_dists)
        if D_max > threshold_ratio * euclidean(centers[0], centers[1]):
            new_center = X[np.argmax(min_dists)]
            centers.append(new_center)
        else:
            break
    return len(centers), np.array(centers)

# AKN: Calculate Initial Weights (Algorithm 2)
def calculate_initial_weights(R, numcl):
    N = (R - R.min(axis=0)) / (R.max(axis=0) - R.min(axis=0) + 1e-10)
    C = N.mean(axis=0)
    dmax = np.max([euclidean(n, C) for n in N])
    X = np.zeros((R.shape[1], numcl))
    for j in range(numcl):
        X[:, j] = C + np.random.uniform(-dmax, dmax, R.shape[1])
    return X.T

# AKN: Clustering (Algorithm 3)
def kohonen_clustering(X, initial_centers, num_epochs=100, initial_lr=0.5, initial_radius=0.5):
    weights = initial_centers.copy()
    num_neurons = len(weights)
    sigma = initial_radius * num_neurons
    sigma_decay = sigma / num_epochs
    for epoch in range(num_epochs):
        lr = initial_lr * np.exp(-epoch / num_epochs)
        sigma = sigma - sigma_decay * epoch
        for x in X:
            distances = euclidean_distances([x], weights).flatten()
            winner_idx = np.argmin(distances)
            for j in range(num_neurons):
                dist_to_winner = np.abs(j - winner_idx)
                influence = np.exp(-dist_to_winner**2 / (2 * sigma**2))
                weights[j] += lr * influence * (x - weights[j])
    assignments = [np.argmin(euclidean_distances([x], weights)) for x in X]
    return assignments, weights

# Apply AKN on original data
X_clustering = X_scaled_data.drop(columns=['DU_ID', 'Label', 'Entropy_DU', 'Pearson_Corr']).values
num_neurons, initial_centers = calculate_optimal_neurons(X_clustering, threshold_ratio=0.5)
initial_weights = calculate_initial_weights(X_clustering, num_neurons)
cluster_labels, final_weights = kohonen_clustering(X_clustering, initial_weights, num_epochs=100)
X_scaled_data['Cluster'] = cluster_labels
print("Clustering Silhouette Score:", silhouette_score(X_clustering, cluster_labels))

# Compute MAD on original data
mean_C = (X_scaled_data['TCP_Packets_mean'] / (X_scaled_data['TCP_Packets_mean'] + 1e-10)).mean()
std_C = (X_scaled_data['TCP_Packets_mean'] / (X_scaled_data['TCP_Packets_mean'] + 1e-10)).std() or 1e-10
mean_H = X_scaled_data['Entropy_DU'].mean()
std_H = X_scaled_data['Entropy_DU'].std() or 1e-10
mean_P = X_scaled_data['Pearson_Corr'].mean()
std_P = X_scaled_data['Pearson_Corr'].std() or 1e-10

mad_scores = []
for _, row in X_scaled_data.iterrows():
    C = (row['TCP_Packets_mean'] / (row['TCP_Packets_mean'] + 1e-10) - mean_C) / std_C
    H = (row['Entropy_DU'] - mean_H) / std_H
    P = (row['Pearson_Corr'] - mean_P) / std_P
    mad = np.sqrt(0.4 * C**2 + 0.3 * H**2 + 0.3 * P**2)
    mad_scores.append(mad if not np.isnan(mad) else 0)
X_scaled_data['MAD'] = mad_scores
X_scaled_data['MAD'] = scaler.fit_transform(X_scaled_data[['MAD']]).flatten()
X_scaled_data['MAD'].fillna(0, inplace=True)

# Balance data for threshold optimization
X_array = X_scaled_data.drop(columns=['DU_ID', 'Label', 'Entropy_DU', 'Pearson_Corr', 'Cluster', 'MAD']).values
y_array = X_scaled_data['Label'].values
smote = SMOTE(random_state=42, k_neighbors=5)
X_resampled, y_resampled = smote.fit_resample(X_array, y_array)
X_scaled_data_resampled = pd.DataFrame(X_resampled, columns=X_scaled_data.drop(columns=['DU_ID', 'Label', 'Entropy_DU', 'Pearson_Corr', 'Cluster', 'MAD']).columns)
X_scaled_data_resampled['Label'] = y_resampled
X_scaled_data_resampled['MAD'] = scaler.transform(X_scaled_data[['MAD']].iloc[:len(X_resampled)].values).flatten()

# Optimize Thresholds
fpr, tpr, thresholds = roc_curve(X_scaled_data_resampled['Label'], X_scaled_data_resampled['MAD'])
optimal_idx = np.argmax(tpr - fpr)
optimal_z = (thresholds[optimal_idx] - X_scaled_data_resampled['MAD'].mean()) / X_scaled_data_resampled['MAD'].std()
LMAD = X_scaled_data_resampled['MAD'].mean() + optimal_z * X_scaled_data_resampled['MAD'].std()
X_scaled_data['Abnormal_DU'] = (X_scaled_data['MAD'] > LMAD).astype(int)

best_ladur = 0.55
best_f1 = 0
for ladur in [0.5, 0.55, 0.6]:
    cluster_votes = X_scaled_data.groupby('Cluster')['Abnormal_DU'].mean()
    abnormal_clusters = cluster_votes[cluster_votes >= ladur].index
    X_scaled_data['Predicted_Label'] = X_scaled_data['Cluster'].apply(lambda c: 1 if c in abnormal_clusters else 0)
    f1 = f1_score(X_scaled_data['Label'], X_scaled_data['Predicted_Label'])
    if f1 > best_f1:
        best_f1 = f1
        best_ladur = ladur

# Final Detection
LMAD = X_scaled_data['MAD'].mean() + optimal_z * X_scaled_data['MAD'].std()
X_scaled_data['Abnormal_DU'] = (X_scaled_data['MAD'] > LMAD).astype(int)
cluster_votes = X_scaled_data.groupby('Cluster')['Abnormal_DU'].mean()
abnormal_clusters = cluster_votes[cluster_votes >= best_ladur].index
X_scaled_data['Predicted_Label'] = X_scaled_data['Cluster'].apply(lambda c: 1 if c in abnormal_clusters else 0)
X_scaled_data['Predicted_Label'].fillna(0, inplace=True)  # Fallback for any remaining NaN
print("Final Detection Evaluation:\n")
print(classification_report(X_scaled_data['Label'], X_scaled_data['Predicted_Label']))

# Cross-validate Random Forest
rf = RandomForestClassifier(random_state=42)
scores = cross_val_score(rf, X_array, X_scaled_data['Label'], cv=5, scoring='f1_weighted')
print("Random Forest Cross-Validation F1 Scores:", scores, "Mean:", scores.mean())

<ipython-input-21-3979325510>:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Protocol'].fillna(data['Protocol'].mode()[0], inplace=True)
<ipython-input-21-3979325510>:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=T

Estimated DU Duration: 2 seconds
Packets per DU:
 count    733.000000
mean      47.553888
std      131.489508
min        2.000000
25%        8.000000
50%       14.000000
75%       24.000000
max      733.000000
dtype: float64


<ipython-input-21-3979325510>:98: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg_data['Entropy_DU'] = data.groupby('DU_ID').apply(compute_du_entropy).reindex(agg_data.index).values
<ipython-input-21-3979325510>:125: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(du_data['TCP_Packets'], du_data['Total_Packets'])


Clustering Silhouette Score: -0.5314425762953277


<ipython-input-21-3979325510>:207: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_scaled_data['MAD'].fillna(0, inplace=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


ValueError: Length of values (733) does not match length of index (1390)